# 05. 오차역전파법

가중치 매개변수의 기울기를 효율적으로 계산하기 위해 **오차역전파법(backpropagation)** 을 이용한다.

## 5.1. 계산 그래프

* 계산 그래프를 사용하면 **국소적 계산** 이 가능하다.
* 중간에 구한 미분 결과를 공유할 수 있어서 다수의 미분을 효율적으로 계산할 수 있다.

## 5.2. 연쇄법칙

* 합성함수의 미분은 합성함수를 구성하는 각 함수의 미분의 곱으로 구할 수 있다.
* 계산 그래프의 역전파는 연쇄법칙의 원리를 따른다.

## 5.3. 역전파

* **덧셈 노드**의 역전파는 입력 신호를 그대로 다음 노드에 출력한다.

* **곱셈 노드**의 역전파는 입력 신호에 순전파 때의 입력값들을 '서로 바꾼 값'을 곱해서 구할 수 있다.
* 곱셈 노드를 구현할 때에는 순전파의 입력 신호를 변수에 저장해둔다.

## 5.4. 단순한 계층 구현하기

* 모든 계층은 forward()와 backward()라는 공통의 메서드(인터페이스)를 갖도록 구현한다.
* forward()는 순전파, backward()는 역전파를 처리한다.

#### 5.4.1. 곱셈 계층

In [2]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y                
        out = x * y

        return out

    def backward(self, dout):
        dx = dout * self.y  # x와 y를 바꾼다.
        dy = dout * self.x

        return dx, dy

In [3]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

# output
print(price) # 220

220.00000000000003


In [4]:
# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

# 갹 변수에 대한 미분
print(dapple, dapple_num, dtax) # 2.2 110 200

2.2 110.00000000000001 200


#### 5.4.2. 덧셈 계층

In [5]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y

        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy

#### buy_apple_orange

In [6]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


## 5.5. 활성화 함수 계층 구현하기

#### 5.5.1. ReLU 계층

In [8]:
import numpy as np

class Relu:
    def __init__(self):
        self.mask = None  # mask : True/False로 구성된 넘파이 배열 (순전파 입력값이 0 이하면 True, 그 외는 False)

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

In [9]:
x = np.array([[1.0, -0.5], [2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [ 2.   3. ]]


In [10]:
 # mask : True/False로 구성된 넘파이 배열 (순전파 입력값이 0 이하면 True, 그 외는 False)
mask = (x <= 0)
print(mask)

[[False  True]
 [False False]]


#### 5.5.2. Sigmoid 계층

In [11]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx